# 📘 Step 2: MLflow Tracing - 자동 추적 (Autolog)

## 목표
- LangChain Autolog 활성화로 원라인 Tracing
- Trace 구조 이해 (Span, Parent-Child 관계)
- Jupyter Notebook에서 Trace 시각화

## 학습 포인트
- Autolog의 편리함
- Trace vs Run의 차이
- Span 구조 (Parent-Child Hierarchy)
- Token usage 자동 추적

## 0. 사전 준비

### ⚠️ 주의사항
이 노트북을 실행하기 전에:
- **Step 0, 1** 완료 권장
- `.env` 파일 설정 완료
- 필요한 패키지 설치 완료

## 1. 환경 설정 및 패키지 임포트

### 1.1 환경 변수 로드

In [41]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

### 1.2 필요한 라이브러리 임포트

In [42]:
import time
import json
from typing import TypedDict, List
from datetime import datetime

# MLflow
import mlflow
from mlflow import MlflowClient
from mlflow.entities import Trace

# LangChain & LangGraph
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END

print("✅ 라이브러리 임포트 완료")
print(f"MLflow 버전: {mlflow.__version__}")

✅ 라이브러리 임포트 완료
MLflow 버전: 3.7.0


## 2. MLflow Tracking 설정

### 2.1 Tracking URI 및 Experiment 설정

In [43]:
# Tracking URI 설정
mlflow.set_tracking_uri("./mlruns")

# Experiment 설정
experiment_name = "rag_agent_tracing"
mlflow.set_experiment(experiment_name)

experiment = mlflow.get_experiment_by_name(experiment_name)

print("✅ MLflow 설정 완료")
print(f"📊 Experiment Name: {experiment.name}")
print(f"🆔 Experiment ID: {experiment.experiment_id}")

✅ MLflow 설정 완료
📊 Experiment Name: rag_agent_tracing
🆔 Experiment ID: 899830758005961363


## 3. LangChain Autolog 활성화

### 3.1 기본 Autolog

**단 한 줄의 코드로 자동 추적 활성화!**

In [44]:
# LangChain 자동 tracing 활성화
mlflow.langchain.autolog()

print("✅ LangChain Autolog 활성화 완료")
print("📝 이제 LangChain/LangGraph 호출이 자동으로 추적됩니다!")

✅ LangChain Autolog 활성화 완료
📝 이제 LangChain/LangGraph 호출이 자동으로 추적됩니다!


### 3.2 세밀한 Autolog 설정 (선택사항)

더 많은 정보를 로깅하려면 추가 옵션 활성화

In [45]:
# 더 상세한 로깅 옵션 (선택적)
mlflow.langchain.autolog(
    log_traces=True,                # Trace 활성화
    silent=False,                   # 경고 및 이벤트 로그 표시
    disable=False,                  # Autolog 활성화
    exclusive=False,                # 사용자 생성 run에도 로깅
)

print("✅ 세밀한 Autolog 설정 완료")

✅ 세밀한 Autolog 설정 완료


## 4. RAG Agent 구성

Step 0, 1과 동일한 RAG Agent를 구성합니다.

### 4.1 샘플 문서 준비

In [46]:
sample_documents = [
    """
    RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.
    RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.
    이 방법은 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.
    """,
    """
    Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.
    FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다.
    Vector Store는 유사도 검색(Similarity Search)을 통해 쿼리와 가장 관련성 높은 문서를 빠르게 찾아냅니다.
    """,
    """
    MLflow Tracing은 LLM 애플리케이션의 실행 과정을 상세히 추적하는 기능입니다.
    각 단계의 입력, 출력, 실행 시간, 토큰 사용량 등을 자동으로 기록합니다.
    Trace는 Span의 계층 구조로 구성되며, 각 Span은 특정 작업을 나타냅니다.
    """,
    """
    Span은 Trace의 구성 요소로, 단일 작업 또는 함수 호출을 나타냅니다.
    Parent Span과 Child Span의 계층 구조를 통해 복잡한 워크플로우를 시각화할 수 있습니다.
    각 Span은 시작 시간, 종료 시간, 입력, 출력, 메타데이터를 포함합니다.
    """,
]

print(f"✅ {len(sample_documents)}개의 샘플 문서 준비 완료")

✅ 4개의 샘플 문서 준비 완료


### 4.2 Vector Store 생성

In [47]:
# AWS 환경 변수 로드
AWS_MODEL_ID = os.getenv("AWS_MODEL_ID")
AWS_EMD_MODEL_ID = os.getenv("AWS_EMD_MODEL_ID")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
)

documents = [Document(page_content=doc.strip()) for doc in sample_documents]
split_docs = text_splitter.split_documents(documents)

# 임베딩 및 Vector Store 생성 (AWS Bedrock)
embeddings = BedrockEmbeddings(
    model_id=AWS_EMD_MODEL_ID,
    region_name=AWS_REGION
)

vector_store = FAISS.from_documents(split_docs, embeddings)

print(f"✅ Vector Store 생성 완료 ({len(split_docs)}개 chunks)")
print(f"📦 임베딩 모델: {AWS_EMD_MODEL_ID}")

✅ Vector Store 생성 완료 (4개 chunks)
📦 임베딩 모델: amazon.titan-embed-text-v2:0


### 4.3 RAG Agent 정의

In [48]:
class RAGState(TypedDict):
    """RAG Agent의 상태"""
    query: str
    retrieved_documents: List[Document]
    context: str
    answer: str
    metadata: dict


def retriever_node(state: RAGState) -> RAGState:
    """문서 검색 노드"""
    retrieved_docs = vector_store.similarity_search(state["query"], k=3)
    state["retrieved_documents"] = retrieved_docs
    state["metadata"]["num_retrieved_docs"] = len(retrieved_docs)
    return state


def generator_node(state: RAGState) -> RAGState:
    """답변 생성 노드"""
    # Context 생성
    context = "\n\n".join([
        f"[문서 {i+1}]\n{doc.page_content}"
        for i, doc in enumerate(state["retrieved_documents"])
    ])
    state["context"] = context
    
    # Prompt 생성
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "당신은 질문에 답변하는 친절한 AI 어시스턴트입니다. 제공된 컨텍스트를 기반으로 정확하고 간결한 답변을 제공하세요."),
        ("human", "컨텍스트:\n{context}\n\n질문: {query}\n\n답변:")
    ])
    
    # LLM 호출 (AWS Bedrock)
    llm = ChatBedrock(
        model_id=AWS_MODEL_ID,
        region_name=AWS_REGION,
        model_kwargs={"temperature": 0.7}
    )
    
    messages = prompt_template.format_messages(context=context, query=state["query"])
    response = llm.invoke(messages)
    
    state["answer"] = response.content
    return state


# LangGraph 구성
workflow = StateGraph(RAGState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("generator", generator_node)
workflow.add_edge(START, "retriever")
workflow.add_edge("retriever", "generator")
workflow.add_edge("generator", END)

rag_agent = workflow.compile()

print("✅ RAG Agent 구성 완료")
print(f"🤖 LLM 모델: {AWS_MODEL_ID}")

✅ RAG Agent 구성 완료
🤖 LLM 모델: global.anthropic.claude-sonnet-4-5-20250929-v1:0


## 5. Autolog로 자동 Tracing 실행

### 5.1 첫 번째 실행 - 자동 Trace 생성

In [49]:
print("="*80)
print("🚀 RAG Agent 실행 (Autolog로 자동 Trace 생성)")
print("="*80)

# 테스트 쿼리
test_query = "MLflow Tracing이 무엇인가요?"

# 초기 상태
initial_state = {
    "query": test_query,
    "retrieved_documents": [],
    "context": "",
    "answer": "",
    "metadata": {}
}

# RAG Agent 실행 (자동으로 trace 생성됨!)
result = rag_agent.invoke(initial_state)

print(f"\n✅ 실행 완료")
print(f"\n❓ 질문: {test_query}")
print(f"\n💬 답변:\n{result['answer']}")
print(f"\n📄 검색된 문서: {len(result['retrieved_documents'])}개")
print("\n" + "="*80)
print("📝 Trace가 자동으로 MLflow에 기록되었습니다!")
print("="*80)

🚀 RAG Agent 실행 (Autolog로 자동 Trace 생성)

✅ 실행 완료

❓ 질문: MLflow Tracing이 무엇인가요?

💬 답변:
MLflow Tracing은 LLM 애플리케이션의 실행 과정을 상세히 추적하는 기능입니다.

주요 특징은 다음과 같습니다:

1. **자동 기록**: 각 단계의 입력, 출력, 실행 시간, 토큰 사용량 등을 자동으로 기록합니다.

2. **계층 구조**: Trace는 Span의 계층 구조로 구성되며, 각 Span은 특정 작업을 나타냅니다.

3. **워크플로우 시각화**: Parent Span과 Child Span의 계층 구조를 통해 복잡한 워크플로우를 시각화할 수 있습니다.

4. **상세 정보**: 각 Span은 시작 시간, 종료 시간, 입력, 출력, 메타데이터를 포함하여 실행 과정을 세밀하게 분석할 수 있게 합니다.

📄 검색된 문서: 3개

📝 Trace가 자동으로 MLflow에 기록되었습니다!


### 5.2 여러 쿼리로 Trace 생성

In [50]:
# 여러 테스트 쿼리
test_queries = [
    "RAG가 무엇인가요?",
    "Vector Store는 어떻게 작동하나요?",
    "Span의 계층 구조에 대해 설명해주세요.",
]

print("\n" + "="*80)
print("🔄 여러 쿼리로 Trace 생성")
print("="*80)

for i, query in enumerate(test_queries, 1):
    print(f"\n[{i}/{len(test_queries)}] 쿼리: {query}")
    
    initial_state = {
        "query": query,
        "retrieved_documents": [],
        "context": "",
        "answer": "",
        "metadata": {}
    }
    
    result = rag_agent.invoke(initial_state)
    print(f"✅ 완료 - 답변 길이: {len(result['answer'])} 문자")

print(f"\n{'='*80}")
print(f"✅ 총 {len(test_queries)}개의 Trace 생성 완료")
print("="*80)


🔄 여러 쿼리로 Trace 생성

[1/3] 쿼리: RAG가 무엇인가요?
✅ 완료 - 답변 길이: 281 문자

[2/3] 쿼리: Vector Store는 어떻게 작동하나요?
✅ 완료 - 답변 길이: 496 문자

[3/3] 쿼리: Span의 계층 구조에 대해 설명해주세요.
✅ 완료 - 답변 길이: 458 문자

✅ 총 3개의 Trace 생성 완료


## 6. Trace 검색 및 분석

### 6.1 최근 Trace 검색

In [51]:
# Trace 검색 (최신 API 사용)
traces = mlflow.search_traces(
    locations=[experiment.experiment_id],  # locations 파라미터 사용 (experiment_ids는 deprecated)
    max_results=10,
    order_by=["timestamp_ms DESC"],  # 최신순 정렬
    return_type="list"  # list 타입으로 반환
)

print(f"\n📊 검색된 Trace: {len(traces)}개\n")
print("="*80)
print(f"{'Request ID':<40} {'실행 시간':<15} {'Spans':<10}")
print("="*80)

for trace in traces:
    request_id = trace.info.request_id[:36]  # 앞 36자만 표시
    exec_time = f"{trace.info.execution_time_ms:.0f}ms" if trace.info.execution_time_ms else "N/A"
    # spans는 search_spans() 메서드로 접근
    num_spans = len(trace.search_spans())
    
    print(f"{request_id:<40} {exec_time:<15} {num_spans:<10}")

print("="*80)


📊 검색된 Trace: 8개

Request ID                               실행 시간           Spans     
tr-343aeaf764e899ccf61b7cd78524af78      5935ms          4         
tr-2269058e15d2ef0e990ce0bdd221137e      5899ms          4         
tr-310e991193d35a586bb5a21dbf0d4516      5139ms          4         
tr-5ce4a10934a606eb5c749ce56d4a56cf      4236ms          4         
tr-10c9234fb6358234747ffa42ec66e3f8      6244ms          4         
tr-6f9c12f6277fb8e5939c2f1264473c3a      6346ms          4         
tr-db01ed48e6e72ae8f27faf7e688e9159      5278ms          4         
tr-c3535b12e726b0837905cc851b683661      4809ms          4         


### 6.2 특정 Trace 상세 분석

In [52]:
if traces:
    # 가장 최근 Trace 선택
    latest_trace = traces[0]
    
    print("\n" + "="*80)
    print("🔍 최근 Trace 상세 정보")
    print("="*80)
    
    print(f"\n📋 기본 정보:")
    print(f"  • Request ID: {latest_trace.info.request_id}")
    print(f"  • Timestamp: {datetime.fromtimestamp(latest_trace.info.timestamp_ms/1000)}")
    print(f"  • 실행 시간: {latest_trace.info.execution_time_ms:.0f}ms")
    print(f"  • 상태: {latest_trace.info.status}")
    
    print(f"\n📊 Spans 구조:")
    # search_spans() 메서드로 spans 가져오기
    spans = latest_trace.search_spans()
    if spans:
        for i, span in enumerate(spans, 1):
            indent = "  " * (span.parent_id is not None)
            span_type = "└─" if span.parent_id else "├─"
            print(f"{indent}{span_type} [{i}] {span.name}")
            print(f"{indent}    • Span ID: {span.span_id[:16]}...")
            print(f"{indent}    • 시작: {span.start_time_ns // 1_000_000}ms")
            print(f"{indent}    • 종료: {span.end_time_ns // 1_000_000}ms")
            if span.parent_id:
                print(f"{indent}    • Parent: {span.parent_id[:16]}...")
    
    print("\n" + "="*80)
else:
    print("⚠️  검색된 Trace가 없습니다.")


🔍 최근 Trace 상세 정보

📋 기본 정보:
  • Request ID: tr-343aeaf764e899ccf61b7cd78524af78
  • Timestamp: 2025-12-16 11:40:58.867000
  • 실행 시간: 5935ms
  • 상태: TraceStatus.OK

📊 Spans 구조:
├─ [1] LangGraph
    • Span ID: 886826f17bedcd23...
    • 시작: 1765852858867ms
    • 종료: 1765852864802ms
  └─ [2] retriever
      • Span ID: 515298ba9da38d9b...
      • 시작: 1765852858868ms
      • 종료: 1765852858969ms
      • Parent: 886826f17bedcd23...
  └─ [3] generator
      • Span ID: 53e44f3d598f722b...
      • 시작: 1765852858970ms
      • 종료: 1765852864801ms
      • Parent: 886826f17bedcd23...
  └─ [4] ChatBedrock
      • Span ID: b4150cf077b361c2...
      • 시작: 1765852859073ms
      • 종료: 1765852864798ms
      • Parent: 53e44f3d598f722b...



### 6.3 Trace 입출력 확인

In [53]:
if traces:
    latest_trace = traces[0]
    spans = latest_trace.search_spans()
    
    if spans:
        print("\n" + "="*80)
        print("📥📤 Trace 입출력 정보")
        print("="*80)
        
        for i, span in enumerate(spans, 1):
            print(f"\n[{i}] {span.name}")
            print("-" * 80)
            
            # Inputs
            if span.inputs:
                print(f"📥 Inputs:")
                inputs_str = str(span.inputs)[:200]  # 처음 200자만
                print(f"  {inputs_str}...")
            
            # Outputs
            if span.outputs:
                print(f"\n📤 Outputs:")
                outputs_str = str(span.outputs)[:200]  # 처음 200자만
                print(f"  {outputs_str}...")
            
            # Attributes (메타데이터)
            if span.attributes:
                print(f"\n🏷️  Attributes:")
                for key, value in list(span.attributes.items())[:5]:  # 처음 5개만
                    print(f"  • {key}: {value}")
        
        print("\n" + "="*80)


📥📤 Trace 입출력 정보

[1] LangGraph
--------------------------------------------------------------------------------
📥 Inputs:
  {'query': 'Span의 계층 구조에 대해 설명해주세요.', 'retrieved_documents': [], 'context': '', 'answer': '', 'metadata': {}}...

📤 Outputs:
  {'query': 'Span의 계층 구조에 대해 설명해주세요.', 'retrieved_documents': [{'id': '6f8ba009-c207-4cdb-ae06-9adb705eb76d', 'metadata': {}, 'page_content': 'Span은 Trace의 구성 요소로, 단일 작업 또는 함수 호출을 나타냅니다.\n    Parent Spa...

🏷️  Attributes:
  • mlflow.traceRequestId: tr-343aeaf764e899ccf61b7cd78524af78
  • mlflow.spanType: CHAIN
  • mlflow.spanInputs: {'query': 'Span의 계층 구조에 대해 설명해주세요.', 'retrieved_documents': [], 'context': '', 'answer': '', 'metadata': {}}
  • mlflow.spanOutputs: {'query': 'Span의 계층 구조에 대해 설명해주세요.', 'retrieved_documents': [{'id': '6f8ba009-c207-4cdb-ae06-9adb705eb76d', 'metadata': {}, 'page_content': 'Span은 Trace의 구성 요소로, 단일 작업 또는 함수 호출을 나타냅니다.\n    Parent Span과 Child Span의 계층 구조를 통해 복잡한 워크플로우를 시각화할 수 있습니다.\n    각 Span은 시작 시간, 종료 시간, 입력, 출력

## 7. Span 계층 구조 시각화

### 7.1 Trace의 Span 계층 구조 트리로 출력

In [54]:
def print_span_tree(trace, indent=0):
    """
    Trace의 Span 계층 구조를 트리 형태로 출력
    """
    # search_spans() 메서드로 spans 가져오기
    spans = trace.search_spans()
    
    if not spans:
        print("Span이 없습니다.")
        return
    
    # Parent-Child 관계 구축
    span_map = {span.span_id: span for span in spans}
    root_spans = [span for span in spans if span.parent_id is None]
    
    def print_span_recursive(span, level=0):
        prefix = "  " * level
        symbol = "└─" if level > 0 else "🌳"
        
        # Span 정보 출력
        duration_ms = (span.end_time_ns - span.start_time_ns) / 1_000_000
        print(f"{prefix}{symbol} {span.name} ({duration_ms:.2f}ms)")
        
        # Span type 표시
        if span.span_type:
            print(f"{prefix}   📋 Type: {span.span_type}")
        
        # Child Spans 재귀 출력
        children = [s for s in spans if s.parent_id == span.span_id]
        for child in children:
            print_span_recursive(child, level + 1)
    
    # Root Spans부터 출력
    for root_span in root_spans:
        print_span_recursive(root_span)
    
    # Trace 전체의 Token usage 표시 (trace.info.token_usage 사용)
    token_usage = trace.info.token_usage
    if token_usage:
        print(f"\n💰 전체 Token Usage:")
        print(f"   • Input Tokens: {token_usage.get('input_tokens', 0)}")
        print(f"   • Output Tokens: {token_usage.get('output_tokens', 0)}")
        print(f"   • Total Tokens: {token_usage.get('total_tokens', 0)}")


if traces:
    print("\n" + "="*80)
    print("🌳 Span 계층 구조")
    print("="*80 + "\n")
    
    print_span_tree(traces[0])
    
    print("\n" + "="*80)


🌳 Span 계층 구조

🌳 LangGraph (5935.31ms)
   📋 Type: CHAIN
  └─ retriever (101.47ms)
     📋 Type: CHAIN
  └─ generator (5831.78ms)
     📋 Type: CHAIN
    └─ ChatBedrock (5724.35ms)
       📋 Type: CHAT_MODEL

💰 전체 Token Usage:
   • Input Tokens: 555
   • Output Tokens: 380
   • Total Tokens: 935



## 8. Token Usage 자동 추적

Autolog는 LLM 호출 시 Token usage를 자동으로 추적합니다.

**MLflow 3.x의 Token Usage 구조:**
- Token 정보는 `trace.info.token_usage` 딕셔너리에 저장됩니다
- 키: `input_tokens`, `output_tokens`, `total_tokens`
- Span 레벨이 아닌 Trace 레벨에서 집계됩니다

### 8.1 Token Usage 추출

In [55]:
def extract_token_usage(trace):
    """
    Trace에서 Token usage 정보 추출
    
    MLflow 3.x에서는 token usage가 trace.info.token_usage에 딕셔너리로 저장됩니다.
    키: 'input_tokens', 'output_tokens', 'total_tokens'
    """
    # trace.info.token_usage에서 직접 가져오기
    token_usage = trace.info.token_usage or {}
    
    return {
        'total_tokens': token_usage.get('total_tokens', 0),
        'prompt_tokens': token_usage.get('input_tokens', 0),  # input_tokens가 prompt_tokens와 동일
        'completion_tokens': token_usage.get('output_tokens', 0)  # output_tokens가 completion_tokens와 동일
    }


if traces:
    print("\n" + "="*80)
    print("💰 Token Usage 분석")
    print("="*80 + "\n")
    
    total_all = 0
    total_prompt = 0
    total_completion = 0
    
    for i, trace in enumerate(traces[:5], 1):  # 최근 5개
        usage = extract_token_usage(trace)
        total_all += usage['total_tokens']
        total_prompt += usage['prompt_tokens']
        total_completion += usage['completion_tokens']
        
        print(f"[{i}] Trace {trace.info.request_id[:16]}...")
        print(f"  • Total Tokens: {usage['total_tokens']}")
        print(f"  • Input Tokens: {usage['prompt_tokens']}")
        print(f"  • Output Tokens: {usage['completion_tokens']}")
        print()
    
    print(f"{'='*80}")
    print(f"📊 총 Token 사용량: {total_all} tokens")
    print(f"   - Input Tokens: {total_prompt}")
    print(f"   - Output Tokens: {total_completion}")
    
    # 비용 추정 (AWS Bedrock Claude Sonnet 4.5 기준)
    # 가격은 리전에 따라 다를 수 있음 (us-east-1 기준)
    input_cost_per_1k = 0.003   # $3 per 1M tokens = $0.003 per 1K
    output_cost_per_1k = 0.015  # $15 per 1M tokens = $0.015 per 1K
    
    input_cost = (total_prompt / 1000) * input_cost_per_1k
    output_cost = (total_completion / 1000) * output_cost_per_1k
    total_cost = input_cost + output_cost
    
    print(f"💵 예상 비용: ${total_cost:.6f}")
    print(f"   - Input: ${input_cost:.6f} ({total_prompt} tokens)")
    print(f"   - Output: ${output_cost:.6f} ({total_completion} tokens)")
    print(f"   - 모델: Claude Sonnet 4.5 (Bedrock)")
    print("="*80)


💰 Token Usage 분석

[1] Trace tr-343aeaf764e89...
  • Total Tokens: 935
  • Input Tokens: 555
  • Output Tokens: 380

[2] Trace tr-2269058e15d2e...
  • Total Tokens: 908
  • Input Tokens: 507
  • Output Tokens: 401

[3] Trace tr-310e991193d35...
  • Total Tokens: 801
  • Input Tokens: 545
  • Output Tokens: 256

[4] Trace tr-5ce4a10934a60...
  • Total Tokens: 849
  • Input Tokens: 548
  • Output Tokens: 301

[5] Trace tr-10c9234fb6358...
  • Total Tokens: 922
  • Input Tokens: 555
  • Output Tokens: 367

📊 총 Token 사용량: 4415 tokens
   - Input Tokens: 2710
   - Output Tokens: 1705
💵 예상 비용: $0.033705
   - Input: $0.008130 (2710 tokens)
   - Output: $0.025575 (1705 tokens)
   - 모델: Claude Sonnet 4.5 (Bedrock)


## 9. Trace vs Run 비교

### 9.1 개념 차이

In [56]:
print("\n" + "="*80)
print("📚 Trace vs Run 비교")
print("="*80)

comparison = [
    ["특징", "Trace", "Run"],
    ["-" * 20, "-" * 35, "-" * 35],
    ["목적", "실행 과정의 상세 추적", "실험 결과 기록"],
    ["단위", "단일 요청/호출", "전체 실험"],
    ["구조", "Span의 계층 구조 (DAG)", "평면 구조 (params + metrics)"],
    ["자동화", "Autolog로 자동 생성", "수동 로깅 필요"],
    ["상세도", "각 단계별 입출력 기록", "전체 요약 메트릭만"],
    ["Token 추적", "자동 추적", "수동 로깅 필요"],
    ["디버깅", "매우 유용 (단계별 확인)", "제한적 (전체 결과만)"],
    ["용도", "프로덕션 모니터링, 디버깅", "실험 비교, 모델 선택"],
]

for row in comparison:
    print(f"{row[0]:<20} | {row[1]:<35} | {row[2]:<35}")

print("\n" + "="*80)
print("💡 결론: Trace와 Run은 상호 보완적!")
print("  • Trace: 각 요청의 실행 과정 추적 (디버깅, 모니터링)")
print("  • Run: 실험 설정 및 결과 비교 (하이퍼파라미터 튜닝)")
print("="*80)


📚 Trace vs Run 비교
특징                   | Trace                               | Run                                
-------------------- | ----------------------------------- | -----------------------------------
목적                   | 실행 과정의 상세 추적                        | 실험 결과 기록                           
단위                   | 단일 요청/호출                            | 전체 실험                              
구조                   | Span의 계층 구조 (DAG)                   | 평면 구조 (params + metrics)           
자동화                  | Autolog로 자동 생성                      | 수동 로깅 필요                           
상세도                  | 각 단계별 입출력 기록                        | 전체 요약 메트릭만                         
Token 추적             | 자동 추적                               | 수동 로깅 필요                           
디버깅                  | 매우 유용 (단계별 확인)                      | 제한적 (전체 결과만)                       
용도                   | 프로덕션 모니터링, 디버깅                      | 실험 비교, 모델 선택                       

💡 결론: Trac

## 10. MLflow UI에서 Trace 확인

### 10.1 UI 실행 및 탐색 가이드

In [57]:
print("\n" + "="*80)
print("🌐 MLflow UI에서 Trace 확인하기")
print("="*80)

print("""
1️⃣ 터미널에서 MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ 브라우저에서 접속:
   http://localhost:5000

3️⃣ Experiment 선택:
   • 'rag_agent_tracing' 실험 클릭

4️⃣ Traces 탭 이동:
   • 상단 메뉴에서 'Traces' 탭 클릭

5️⃣ 개별 Trace 확인:
   • Request ID 클릭
   • Span 계층 구조 시각화 확인
   • 각 Span의 입출력 확인
   • Token usage 확인
   • 실행 시간 Waterfall 차트 확인

6️⃣ Trace 필터링:
   • 실행 시간으로 필터링
   • 상태로 필터링 (성공/실패)
   • 날짜 범위로 필터링
""")

print("="*80)


🌐 MLflow UI에서 Trace 확인하기

1️⃣ 터미널에서 MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ 브라우저에서 접속:
   http://localhost:5000

3️⃣ Experiment 선택:
   • 'rag_agent_tracing' 실험 클릭

4️⃣ Traces 탭 이동:
   • 상단 메뉴에서 'Traces' 탭 클릭

5️⃣ 개별 Trace 확인:
   • Request ID 클릭
   • Span 계층 구조 시각화 확인
   • 각 Span의 입출력 확인
   • Token usage 확인
   • 실행 시간 Waterfall 차트 확인

6️⃣ Trace 필터링:
   • 실행 시간으로 필터링
   • 상태로 필터링 (성공/실패)
   • 날짜 범위로 필터링



### 10.2 Jupyter에서 Trace 시각화 (선택사항)

In [58]:
# Jupyter Notebook에서 Trace 정보 간단히 표시
if traces:
    from IPython.display import JSON, display
    
    print("\n📊 Trace JSON 데이터 (최근 Trace)\n")
    
    latest_trace = traces[0]
    spans = latest_trace.search_spans()
    
    # Trace를 딕셔너리로 변환
    trace_dict = {
        "request_id": latest_trace.info.request_id,
        "timestamp": str(datetime.fromtimestamp(latest_trace.info.timestamp_ms/1000)),
        "execution_time_ms": latest_trace.info.execution_time_ms,
        "status": latest_trace.info.status,
        "num_spans": len(spans),
        "spans": [
            {
                "name": span.name,
                "duration_ms": (span.end_time_ns - span.start_time_ns) / 1_000_000,
                "has_parent": span.parent_id is not None
            }
            for span in spans
        ]
    }
    
    # JSON 형태로 표시
    display(JSON(trace_dict, expanded=True))


📊 Trace JSON 데이터 (최근 Trace)



<IPython.core.display.JSON object>

## 11. 고급: Trace 필터링 및 검색

### 11.1 조건별 Trace 검색

In [59]:
# 실행 시간이 긴 Trace 찾기
slow_traces = mlflow.search_traces(
    locations=[experiment.experiment_id],  # locations 파라미터 사용
    filter_string="execution_time_ms > 2000",  # 2초 이상
    max_results=5,
    return_type="list"
)

print(f"\n🐌 느린 Trace ({len(slow_traces)}개):")
for trace in slow_traces:
    print(f"  • {trace.info.request_id[:16]}... - {trace.info.execution_time_ms:.0f}ms")


# 최근 1시간 내 Trace
import time
one_hour_ago = int((time.time() - 3600) * 1000)  # milliseconds

recent_traces = mlflow.search_traces(
    locations=[experiment.experiment_id],  # locations 파라미터 사용
    filter_string=f"timestamp_ms > {one_hour_ago}",
    max_results=10,
    return_type="list"
)

print(f"\n🕐 최근 1시간 Trace ({len(recent_traces)}개)")


🐌 느린 Trace (5개):
  • tr-343aeaf764e89... - 5935ms
  • tr-2269058e15d2e... - 5899ms
  • tr-310e991193d35... - 5139ms
  • tr-5ce4a10934a60... - 4236ms
  • tr-10c9234fb6358... - 6244ms

🕐 최근 1시간 Trace (4개)


## 12. 요약 및 다음 단계

### ✅ 완료한 작업
1. LangChain Autolog 활성화 (단 한 줄!)
2. RAG Agent 실행으로 자동 Trace 생성
3. Trace 구조 이해 (Span 계층)
4. Token usage 자동 추적
5. Trace 검색 및 분석
6. Trace vs Run 개념 비교

### 📊 주요 학습 내용
- **Autolog**: 코드 수정 없이 자동 추적
- **Trace**: 단일 요청의 전체 실행 과정
- **Span**: Trace를 구성하는 개별 작업 단위
- **계층 구조**: Parent-Child 관계로 복잡한 워크플로우 시각화
- **자동 메트릭**: Token usage, 실행 시간 등 자동 수집
- **Token Usage**: `trace.info.token_usage`에서 input/output/total tokens 확인

### 🔧 MLflow 3.x API 주요 변경사항
- `experiment_ids` → `locations` 파라미터로 변경
- `return_type="list"` 명시 필요
- Spans 접근: `trace.data.spans` → `trace.search_spans()` 메서드
- Token usage: Span attributes 대신 `trace.info.token_usage` 딕셔너리 사용

### 🎯 다음 단계 (Step 3)
다음 노트북에서는 **수동 커스텀 Span**을 활용하여:
- RAG 파이프라인의 각 단계를 세밀하게 추적
- 커스텀 메타데이터 추가
- 성능 병목 지점 식별
- 프로덕션 모니터링 준비

→ `03_mlflow_tracing_custom_spans.ipynb`로 계속하세요!

### 💡 Autolog의 장점
1. **Zero-code 추적**: 기존 코드 수정 없음
2. **자동 메트릭**: Token, latency 등 자동 수집
3. **프레임워크 지원**: LangChain, LlamaIndex, DSPy 등
4. **프로덕션 준비**: 비동기 로깅 지원

## 📚 참고 자료

- [MLflow Tracing Documentation](https://www.mlflow.org/docs/2.21.3/tracing/)
- [LangChain Autolog](https://www.mlflow.org/docs/2.21.3/llms/langchain/)
- [Tracing Concepts](https://www.mlflow.org/docs/2.21.3/tracing/#tracing-concepts)
- [OpenTelemetry](https://opentelemetry.io/)